In [ ]:
import sys, os
import datetime
import io
import json
import urllib
import zipfile

import pytz

import numpy
import pandas
import requests

# from common import download_zipped_df, download_path, load_oh_data
from common2 import load_data, download_path

In [ ]:
pandas.set_option('display.max_rows', 1000)

In [ ]:
uri = "https://dshs.texas.gov/coronavirus/TexasCOVID19DailyCountyFatalityCountData.xlsx"
df = pandas.read_excel(uri, skiprows=2)
# df = df[df['County Name'] == 'Total'].copy()
df = df.iloc[-1:, :].copy()
num_cols = len(df.columns)
df = df.iloc[0, 1:]
index = pandas.period_range('2020-03-07', periods=len(df), freq='D')
df = pandas.DataFrame([float(x) for x in df.values], index=index).reset_index()
df.columns = ['Date', 'Deaths']
df.Deaths = df.Deaths.fillna(method='ffill').fillna(0.)
df

In [ ]:
uri = "https://coronavirus.ohio.gov/static/dashboards/COVIDDeathData_CountyOfDeath.csv"
oh = pandas.read_csv(uri, low_memory=False)
oh = oh[['Date Of Death', 'Death Due To Illness Count - County Of Death']].copy()
oh.columns = ['Date', 'Daily']
oh.Daily = [int(d) for d in oh.Daily]
oh = oh[(oh.Daily > 0) & (oh.Daily < 100)].copy()
oh.Date = [pandas.Period(x, freq='D') for x in oh.Date]
df = oh.groupby('Date').sum()[['Daily']].sort_index()
df['Deaths'] = df.Daily.cumsum()
oh.Daily.sum(), oh.Date.dropna().max(), df.tail()

In [ ]:
df3 = oh.groupby('Date').sum()[['Daily']].sort_index()
df3['Deaths'] = df3.Daily.cumsum()
df3

In [ ]:
uri = ("https://results.mo.gov/t/COVID19/views/COVID-19DataforDownload/MetricsbyDateofDeath.csv")
raw = pandas.read_csv(uri).iloc[1:-1, :]
mo = raw.copy()
col = 'Measure Values' if 'Measure Values' in mo.columns else 'Confirmed Deaths'
# print(mo.columns)
mo = mo[['Dod', 'Measure Values']].copy()
mo.columns = ['Date', 'Deaths']
mo.Date = [pandas.Period(str(v), freq='D') for v in mo.Date]
mo = mo[mo.Date >= pandas.Period('2020-01-01', freq='D')].set_index('Date').sort_index()
mo.Deaths = [int(x) for x in mo.Deaths]
mo.Deaths = mo.Deaths.cumsum()
mo

In [ ]:
raw

In [ ]:
all_dates = pandas.period_range(start='2020-01-01', end=mo.index[-1], freq='D')
mo = mo.reindex(all_dates, method='ffill').fillna(0.0).reset_index()
mo.columns = ['Date', 'Deaths']

In [ ]:
uri = "C:\\Users\\prusk\\Downloads\\Nevada Dashboard Extract2.xlsx"
nv = pandas.read_excel(uri, sheet_name='Deaths').iloc[:, [0, 1, 2]]
nv

In [ ]:
nv.columns = ['Date', 'Daily', 'Deaths']
nv.Date = [pandas.Period(v, freq='D') for v in nv.Date]
nv = nv.sort_values('Date')
nv.tail(40)

In [ ]:
uri = ("https://public.tableau.com/vizql/w/NCDHHS_COVID-19_DataDownload/v/DailyCasesandDeathsMetrics/vudcsv"
       "/sessions/37B9DB62168E442FA0310335A1D650EA-0:0/views/13567890650596151931_671940180619205164?summary=true")
nc = pandas.read_csv('C:\\Users\\prusk\\Downloads\\TABLE_DAILY_CASE&DEATHS_METRICS_data.csv')

# nc = nc.set_index('Date').sort_index()
# nc['Deaths'] = nc['Measure Values'].fillna(0.0).cumsum()
# nc = nc.reset_index()[['Date', 'Deaths']].copy()
nc

In [ ]:
url = ("https://services1.arcgis.com/CY1LXxl9zlJeBuRZ/ArcGIS/rest/services/"
       "Florida_COVID_19_Deaths_by_Day/FeatureServer/0/query?"
       "where=ObjectId>0&objectIds=&time=&resultType=standard&outFields=*&returnIdsOnly=false"
       "&returnUniqueIdsOnly=false&returnCountOnly=false&returnDistinctValues=false&cacheHint=false"
       "&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset="
       "&resultRecordCount=&sqlFormat=none&f=pjson&token=")
# User-Agent spoofing is required, if we use the default ("python-requests/x.x.x") the server returns
# an empty 'features'!
rows = requests.get(url, headers={'User-Agent': 'curl/7.64.0'}, verify=False).json()['features']


In [ ]:
df = pandas.DataFrame([row['attributes'] for row in rows])
df

In [ ]:
df['Date'] = [pandas.Period(datetime.datetime.fromtimestamp(d // 1000), freq='D') for d in df.Date1]
df = df.sort_values('Date')
df

In [ ]:
def make_date(r):
    dt = datetime.date(int(r['Year']), int(r['Month']), int(r['Day']))
    return pandas.Period(dt, freq='D')

uri = ("https://myhealthycommunity.dhss.delaware.gov/locations/state/download_covid_19_data/overview")
raw = pandas.read_csv(uri)

df = raw[(raw['Date used'] == 'Date of death') & (raw.Unit == 'people')][['Year', 'Month', 'Day', 'Value']]
df['Date'] = df.apply(make_date, axis=1)
df = df[['Date', 'Value']]
df.columns = ['Date', 'Deaths']
df

In [ ]:
spaz = df.sort_values('Date').copy()
spaz.head(60)

In [ ]:
earliest_date = pandas.Period('2020-03-01', freq='D')
latest_date, meta, cdc_stats, ctp_stats, dod_stats = load_data('2020-03-01', None)

In [ ]:
cdc_stats

In [ ]:
def read_cdc_csv(uri, meta, earliest_date, latest_date):
    df = pandas.read_csv(uri, parse_dates=['Week Ending Date'])
    df = df[(df.Group == 'By Week') & ~df.State.isin(['United States', 'Puerto Rico'])]
    df = df[['Week Ending Date', 'State', 'COVID-19 Deaths']].copy()
    df.columns = ['Date', 'State', 'Weekly']
    df.Date = [pandas.Period(str(v), freq='D') for v in df.Date]
    df = df.sort_values(['State', 'Date'])
    
    all_dates = pandas.period_range(start=df.Date.min()-6, end=df.Date.max(), freq='D')

    state_dfs = list()
    for state, st_df in df.groupby('State'):
        st_df = st_df.set_index('Date')
        st_df.Weekly = st_df.Weekly.fillna(0.)
        st_df['Daily'] = st_df.Weekly / 7.
        st_df = st_df.reindex(all_dates)
        st_df.State = state
        st_df.Daily = st_df.Daily.fillna(method='bfill')
        st_df = st_df.reset_index()
        st_df.Daily = st_df.Daily.rolling(window=13, center=True, win_type='triang', min_periods=1).mean()
        state_dfs.append(st_df)

    df = pandas.concat(state_dfs)
    df.columns = ['Date', 'State', 'Weekly', 'Daily']

    if earliest_date:
        df = df[df.Date >= earliest_date]
    if latest_date:
        df = df[df.Date <= latest_date]

    # Pull in the statistics for states
    df = df.set_index(['State', 'Date']).sort_index()
    df = df.join(meta.set_index('State')[['ST']])

    # Change to using two-letter state abbreviations
    df = df.reset_index()[['ST', 'Date', 'Daily']].copy()

    return df

In [ ]:
foo = read_cdc_csv(download_path('cdc_dod_data.csv'), meta, earliest_date, latest_date)

In [ ]:
_ = foo.groupby('Date').sum()[['Daily']].plot(figsize=(15,5), ylim=0)

In [ ]:
foo.tail()

In [ ]:
uri = "https://data.cdc.gov/api/views/r8kw-7aab/rows.csv?accessType=DOWNLOAD"
df = pandas.read_csv(uri, parse_dates=['Data as of', 'Start Date', 'End Date', 'Week Ending Date'])
df = df[df.Group == 'By Week'][['Week Ending Date', 'State', 'COVID-19 Deaths']].copy()
df.columns = ['Date', 'State', 'Weekly']
df['Deaths'] = df.Weekly.cumsum()

In [ ]:
df[df.State == 'United States'].Weekly.sum(), df[df.State != 'United States'].Weekly.sum()

In [ ]:
# foo = states.loc['CA', :].loc['2020-07-25':, :][['NewG', 'Tot']].copy()
# foo.columns = ['New', 'Current']
# fam = foo.plot(
#     title="New and Current Hospitalizations", secondary_y='Current', figsize=(15,5), ylim=0,
# )
# axes = fam.get_figure().get_axes()
# for i in range(len(axes)):
#     axes[i].set_ylim(0)

In [ ]:
uri = ("https://myhealthycommunity.dhss.delaware.gov/locations/state/download_covid_19_data/overview")
raw = pandas.read_csv(uri)

def make_date(r):
    dt = datetime.date(int(r['Year']), int(r['Month']), int(r['Day']))
    return pandas.Period(dt, freq='D')

df = raw[(raw['Date used'] == 'Date of death') & (raw.Unit == 'people')][['Year', 'Month', 'Day', 'Value']]
df['Date'] = df.apply(make_date, axis=1)
df = df[['Date', 'Value']]
df.columns = ['Date', 'Deaths']
df

In [ ]:
uri = ("/Users/patrick/Downloads/cdc_hospital.csv")
raw = pandas.read_csv(uri, parse_dates=['date'])

In [ ]:
raw.columns

In [ ]:
df = raw[['state', 'date',
          'previous_day_admission_adult_covid_confirmed',
          'previous_day_admission_adult_covid_suspected',
          'previous_day_admission_pediatric_covid_confirmed',
          'previous_day_admission_pediatric_covid_suspected',
          'total_adult_patients_hospitalized_confirmed_and_suspected_covid',
          'total_pediatric_patients_hospitalized_confirmed_and_suspected_covid',
         ]].copy()
df.columns = ['State', 'Date',
              'NewAConf', 'NewASusp', 'NewPConf', 'NewPSusp',
              'TotA', 'TotP']
df = df[df.Date >= '2020-07-25'].copy()
df = df[~df.State.isin(['PR', 'VI'])]
df['NewConf'] = df.NewAConf + df.NewPConf
df['NewSusp'] = df.NewASusp + df.NewPSusp
df['New'] = df.NewConf + df.NewSusp
df['NewG'] = df.NewConf + (df.NewSusp * 0.2)
df['Tot'] = df.TotA + df.TotP

cols = ['NewConf', 'NewSusp', 'New', 'NewG', 'Tot']
us = df.groupby('Date').sum()[cols].sort_index().copy()

states = []
for st, st_df in df.groupby('State'):
    st_df = st_df.set_index('Date').sort_index().copy()
    st_df.NewConf = st_df.NewConf.rolling(window=13, center=True, win_type='triang', min_periods=1).mean()
    st_df.NewSusp = st_df.NewSusp.rolling(window=13, center=True, win_type='triang', min_periods=1).mean()
    st_df.New = st_df.New.rolling(window=13, center=True, win_type='triang', min_periods=1).mean()
    st_df.NewG = st_df.NewG.rolling(window=13, center=True, win_type='triang', min_periods=1).mean()
    st_df.NewG = st_df.NewG.rolling(window=13, center=True, win_type='triang', min_periods=1).mean()
    st_df.Tot = st_df.Tot.rolling(window=13, center=True, win_type='triang', min_periods=1).mean()
    states.append(st_df.reset_index())
states = pandas.concat(states).set_index(['State', 'Date']).sort_index()

us.NewConf = us.NewConf.rolling(window=13, center=True, win_type='triang', min_periods=1).mean()
us.NewSusp = us.NewSusp.rolling(window=13, center=True, win_type='triang', min_periods=1).mean()
us.New = us.New.rolling(window=13, center=True, win_type='triang', min_periods=1).mean()
us.NewG = us.NewG.rolling(window=13, center=True, win_type='triang', min_periods=1).mean()
us.NewG = us.NewG.rolling(window=13, center=True, win_type='triang', min_periods=1).mean()
us.Tot = us.Tot.rolling(window=13, center=True, win_type='triang', min_periods=1).mean()

states

In [ ]:
foo = states.loc['CA', :].loc['2020-07-25':, :][['NewG', 'Tot']].copy()
foo.columns = ['New', 'Current']
fam = foo.plot(
    title="New and Current Hospitalizations", secondary_y='Current', figsize=(15,5), ylim=0,
)
axes = fam.get_figure().get_axes()
for i in range(len(axes)):
    axes[i].set_ylim(0)

In [ ]:
us.loc['2021-01-05':, :]

In [ ]:
uri = ("https://results.mo.gov/t/COVID19/views/COVID-19DataforDownload/MetricsbyDateofDeath.csv")
df = pandas.read_csv(uri).iloc[1:-1, :]
df.tail(2)

In [ ]:
col = 'Measure Values' if 'Measure Values' in mo.columns else 'Confirmed Deaths'
# print(mo.columns)
mo = mo[['Dod', 'Measure Values']].copy()
mo.columns = ['Date', 'Deaths']
mo.Date = [pandas.Period(str(v), freq='D') for v in mo.Date]
mo = mo[mo.Date >= pandas.Period('2020-01-01', freq='D')].set_index('Date').sort_index()
mo.Deaths = [int(x) for x in mo.Deaths]
mo.Deaths = mo.Deaths.cumsum()
mo

In [ ]:
all_dates = pandas.period_range(start='2020-01-01', end=mo.index[-1], freq='D')
mo = mo.reindex(all_dates, method='ffill').fillna(0.0).reset_index()
mo.columns = ['Date', 'Deaths']

In [ ]:
uri = './DateOfDeath.xlsx'
sc = pandas.read_excel(uri, sheet_name='South Carolina', parse_dates=['Date'])
sc = sc[['Date', 'Deaths']].copy()
sc

In [ ]:
uri = "https://www.health.nd.gov/sites/www/files/documents/Files/MSS/coronavirus/charts-data/PublicUseData.csv"
df = pandas.read_csv(uri, parse_dates=['Date'])[['Date', 'Total Deaths']].groupby('Date').sum().sort_index()
__ = df.plot(figsize=(15, 5))

In [ ]:
df.head(30)

In [ ]:
url = ("https://services1.arcgis.com/CY1LXxl9zlJeBuRZ/ArcGIS/rest/services/"
       "Florida_COVID_19_Deaths_by_Day/FeatureServer/0/query?"
       "where=ObjectId>0&objectIds=&time=&resultType=standard&outFields=*&returnIdsOnly=false"
       "&returnUniqueIdsOnly=false&returnCountOnly=false&returnDistinctValues=false&cacheHint=false"
       "&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset="
       "&resultRecordCount=&sqlFormat=none&f=pjson&token=")
# User-Agent spoofing is required, if we use the default ("python-requests/x.x.x") the server returns
# an empty 'features'!
rows = requests.get(url, headers={'User-Agent': 'curl/7.64.0'}).json()['features']
df = pandas.DataFrame([row['attributes'] for row in rows])
df.Date = [pandas.Period(datetime.datetime.fromtimestamp(d // 1000), freq='D') for d in df.Date]
df = df.sort_values('Date')


In [ ]:
df.tail(30)

In [ ]:
datetime.datetime.fromtimestamp(1609390800)

In [ ]:
datetime.datetime.fromtimestamp(df.iloc[0, 0] // 1000)

In [ ]:
ET = pytz.timezone('America/New_York')

In [ ]:
uri = download_path('Case_Data_Parallel.csv')
fl = pandas.read_csv(uri, parse_dates=['Case1', 'EventDate', 'ChartDate'])
fl

In [ ]:
uri = download_path('Case_Data_Parallel.csv')
fl = pandas.read_csv(uri, parse_dates=['Case1', 'EventDate', 'ChartDate'])
fl = fl[['Died', 'Case1', 'EventDate', 'ChartDate']].copy()
fl

In [ ]:
df = fl[fl.Died.isin(('Yes', 'Recent'))][['Case1', 'EventDate', 'ChartDate']].copy()
df['Max'] = df.Case1.combine(df.EventDate.combine(df.ChartDate, max), max)
df['Date'] = [pandas.Period(t.tz_convert(ET), freq='D') for t in df.Max]
df['Spaz'] = 1.0
df

In [ ]:
df = df.groupby('Date').count().sort_index().cumsum()
df.tail()

In [ ]:
all_dates = pandas.period_range(start='2020-01-01', end=df.index[-1], freq='D')
df = df.reindex(all_dates, method='ffill').fillna(0.0)[['Spaz']].reset_index()
df.columns = ['Date', 'Deaths']

In [ ]:
df['Daily'] = (df.Deaths - df.Deaths.shift())

In [ ]:
df.tail(30)

In [ ]:
uri = ("https://results.mo.gov/t/COVID19/views/COVID-19DataforDownload/MetricsbyDateofDeath.csv")
mo = pandas.read_csv(uri).iloc[1:-1, :]
# print(mo.columns)
mo = mo[['Dod', 'Measure Values']].copy()
mo.columns = ['Date', 'Deaths']
mo.Date = [pandas.Period(str(v), freq='D') for v in mo.Date]
mo = mo[mo.Date >= pandas.Period('2020-01-01', freq='D')].set_index('Date').sort_index()
mo.Deaths = [int(x) for x in mo.Deaths]
mo.Deaths = mo.Deaths.cumsum()
all_dates = pandas.period_range(start='2020-01-01', end=mo.index[-1], freq='D')
mo = mo.reindex(all_dates, method='ffill').fillna(0.0).reset_index()
mo.columns = ['Date', 'Deaths']

In [ ]:
mo

In [ ]:
uri = ("https://results.mo.gov/t/COVID19/views/COVID-19DataforDownload/MetricsbyDateofDeath.csv")
mo = pandas.read_csv(uri).iloc[1:-1, :][['Dod', 'Confirmed Deaths']].copy()
mo.columns = ['Date', 'Deaths']
mo.Date = [pandas.Period(str(v), freq='D') for v in mo.Date]
mo.Deaths = [int(x) for x in mo.Deaths]
mo

In [ ]:
mo.Deaths.sum()

In [ ]:
uri = ("https://results.mo.gov/t/COVID19/views/COVID-19DataforDownload/MetricsbyDateofDeath.csv")
pandas.read_csv(uri).iloc[-1:, :]

In [ ]:
uri = download_path('CA_Deaths.csv')
ca = pandas.read_csv(uri, parse_dates=['date'])
ca.date = [pandas.Period(x, freq='D') for x in ca.date]
ca.groupby('date').sum().tail(30)

In [ ]:
uri = './DateOfDeath.xlsx'
az = pandas.read_excel(uri, sheet_name='Arizona', parse_dates=['Date'])
az = az[['Date', 'Deaths']].copy()
az

In [ ]:
uri = "/Users/patrick/Downloads/COVID-19 Rhode Island Data.xlsx"
pandas.read_excel(uri, sheet_name='Trends', parse_dates=['Date'])[['Date', 'Date of death']].copy()

In [ ]:
uri = ("https://data.ct.gov/api/views/abag-bjkj/rows.csv?accessType=DOWNLOAD")
pandas.read_csv(uri, parse_dates=['Date of death'])[['Date of death', 'Total deaths']].sort_values('Date of death')


In [ ]:
uri = ("https://results.mo.gov/t/COVID19/views/COVID-19DataforDownload/MetricsbyDateofDeath.csv")
mo = pandas.read_csv(uri).iloc[1:-1, :][['Dod', 'Measure Values']].copy()
mo.columns = ['Date', 'Daily']
mo.Date = [pandas.Period(str(v), freq='D') for v in mo.Date]
mo = mo[mo.Date >= pandas.Period('2020-01-01', freq='D')]
mo.Daily = [int(x) for x in mo.Daily]
mo['Deaths'] = mo.Daily.cumsum()
mo.head()

In [ ]:
uri = ("https://data.virginia.gov/api/views/9d6i-p8gz/rows.csv?accessType=DOWNLOAD")
va = pandas.read_csv(uri, parse_dates=['Event Date'])[['Event Date', 'Number of Deaths']].copy()
va = va.groupby('Event Date').sum().reset_index().sort_values('Event Date')
va.columns = ['Date', 'Daily']
va['Deaths'] = va.Daily.cumsum()
va.tail(40)

In [ ]:
uri = "/Users/patrick/Downloads/Nevada Dashboard Extract.xlsx"
nv = pandas.read_excel(uri, sheet_name='Deaths', skiprows=2).iloc[:, [0, 1, 2]].copy()
nv.columns = ['Date', 'Daily', 'Deaths']
unknown_deaths = nv[nv.Date == 'Unknown'].Daily.iloc[0]
nv = nv[nv.Date != 'Unknown'].copy()
nv.Date = [pandas.Period(v, freq='D') for v in nv.Date]
nv = nv.sort_values('Date')
total_deaths = nv.iloc[-1, -1]
nv.Deaths = nv.Deaths * ((total_deaths + unknown_deaths) / total_deaths)
nv.tail(40)

In [ ]:
uri = ("https://opendata.arcgis.com/datasets/859602b5d427456f821de3830f89301c_11.csv?"
       "outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D")
wi = pandas.read_csv(uri, parse_dates=['DATE'])[['DATE', 'DEATHS']].copy()
wi.columns = ['Date', 'Deaths']
wi.Date = [pandas.Period(v.date(), freq='D') for v in wi.Date]
wi = wi.sort_values('Date')
wi['Daily'] = wi.Deaths - wi.Deaths.shift()
wi.tail(40)

In [ ]:
uri = ("https://hub.mph.in.gov/dataset/6bcfb11c-6b9e-44b2-be7f-a2910d28949a/resource/"
       "7661f008-81b5-4ff2-8e46-f59ad5aad456/download/covid_report_death_date_agegrp.xlsx")
in_ = pandas.read_excel(uri, parse_dates=['date'])[['date', 'covid_deaths']].copy()
in_ = in_.groupby('date').sum().reset_index()
in_['Deaths'] = in_.covid_deaths.cumsum()
in_.columns = ['Date', 'Daily', 'Deaths']
in_.tail(40)

In [ ]:
uri = ("https://www.michigan.gov/documents/coronavirus/"
       "Cases_and_Deaths_by_County_and_by_Date_of_Symptom_Onset_or_by_Date_of_Death2021-01-18_713236_7.xlsx")
mi = pandas.read_excel(uri, parse_dates=['Date'])
mi

In [ ]:
df = mi.groupby('Date').sum()[['Deaths.Cumulative']].reset_index()
df.columns = ['Date', 'Deaths']
df.tail(40)

In [ ]:
uri = "https://www.mass.gov/doc/covid-19-raw-data-january-19-2021/download"
ma = pandas.read_excel(uri)
ma

In [ ]:
uri = ("https://www.tn.gov/content/dam/tn/health/documents/cedep/novel-coronavirus"
       "/datasets/Public-Dataset-Daily-Case-Info.XLSX")
tn = pandas.read_excel(uri, parse_dates=['DATE'])[['DATE', 'TOTAL_DEATHS_BY_DOD']].copy()
tn.columns = ['Date', 'Deaths']
tn

In [ ]:
uri = ("https://public.tableau.com/vizql/w/NCDHHS_COVID-19_DataDownload/v/DailyCasesandDeathsMetrics/vudcsv"
       "/sessions/37B9DB62168E442FA0310335A1D650EA-0:0/views/13567890650596151931_671940180619205164?summary=true")
nc = pandas.read_csv('/Users/patrick/Downloads/TABLE_DAILY_CASE&DEATHS_METRICS_data.csv',
                     encoding='utf_16', sep='\t', parse_dates=['Date'])[['Date', 'Measure Values']]
nc = nc.set_index('Date').sort_index()
nc['Deaths'] = nc['Measure Values'].fillna(0.0).cumsum()
nc = nc.reset_index()[['Date', 'Deaths']].copy()
nc.tail(20)

In [ ]:
nc

In [ ]:
fl = pandas.read_csv('/Users/patrick/Downloads/Florida_COVID19_Case_Line_Data.csv', parse_dates=['EventDate'])

In [ ]:
df = fl[['Died', 'EventDate']].copy()
df = df[df.Died.isin(('Yes', 'Recent'))][['EventDate']].copy()
df['Date'] = [pandas.Period(d.date(), freq='D') for d in df.EventDate]
df = df.groupby('Date').count().sort_index().cumsum()
all_dates = pandas.period_range(start='2020-01-01', end=df.index[-1], freq='D')
df = df.reindex(all_dates, method='ffill').fillna(0.0).reset_index()
df.columns = ['Date', 'Deaths']
df.head()

In [ ]:
df = load_fl_data()

In [ ]:
df['Daily'] = (df.Deaths - df.Deaths.shift())
df.tail(30)

In [ ]:
pandas.Period(df.iloc[0, 0].date(), freq='D')

In [ ]:
uri = ("https://services1.arcgis.com/CY1LXxl9zlJeBuRZ/arcgis/rest/services/"
       "Florida_COVID19_Case_Line_Data_NEW/FeatureServer/0/query?"
       "where=Died%20%3D%20%27YES%27%20OR%20Died%20%3D%20%27RECENT%27&"
       "outFields=EventDate&returnGeometry=false&outSR=4326&f=json")
uri

In [ ]:
response = urllib.request.urlopen(uri)
json_bytes = response.read()
json_data = json.loads(json_bytes)
len(json_data['features'])

In [ ]:
# 10/27/2020, 12:27 PM   is 1603816045000
import datetime
print(datetime.datetime.fromtimestamp(1603816045))

In [ ]:
uri = "https://data.pa.gov/api/views/fbgu-sqgp/rows.csv?accessType=DOWNLOAD&bom=true&format=true"
df = pandas.read_csv(uri, parse_dates=['Date of Death'])
df = df[df['County Name'] == 'Pennsylvania']
df = df[['Date of Death', 'Total Deaths']].copy()
df.columns = ['Date', 'Deaths']
df = df.sort_values('Date')
df.Date = [pandas.Period(str(v), freq='D') for v in df.Date]
pre_dates = pandas.period_range('2020-03-01', periods=17, freq='D')
df2 = pandas.DataFrame(0.0, index=pre_dates, columns=['Deaths']).reset_index()
df2.columns = ['Date', 'Deaths']
final = pandas.concat([df2, df]).sort_values('Date')
final.tail(20)

In [ ]:
uri = "https://dshs.texas.gov/coronavirus/TexasCOVID19DailyCountyFatalityCountData.xlsx"
df = pandas.read_excel(uri, skiprows=2)
df = df[df['County Name'] == 'Total'].copy()
num_cols = len(df.columns)
df = df.iloc[0, 1:]
index = pandas.period_range('2020-03-07', periods=len(df), freq='D')
df = pandas.DataFrame(df.values, index=index).reset_index()
df.columns = ['Date', 'Deaths']


In [ ]:
georgia_uri = "https://ga-covid19.ondemand.sas.com/docs/ga_covid_data.zip"

In [ ]:
df = download_zipped_df(georgia_uri, 'epicurve_symptom_date.csv', parse_dates=['symptom date'])

In [ ]:
df = df[df.measure == 'state_total'][['symptom date', 'cases', 'deaths']].copy()
df.columns = ['Date', 'Cases', 'Deaths']
df = df.set_index('Date')
df

In [ ]:
raise ValueError()

In [ ]:
dash_uri_pattern = "https://www.mass.gov/doc/covid-19-dashboard-%s/download"
raw_uri_pattern = "https://www.mass.gov/doc/covid-19-raw-data-%s/download"

In [ ]:
dt = (datetime.datetime.now() - datetime.timedelta(hours=16)).date()
dt = dt.strftime("%b-%d-%Y").lower()
dash_uri = dash_uri_pattern % dt
raw_uri = raw_uri_pattern % dt
print(dash_uri)
print(raw_uri)


In [ ]:
buffer = requests.get(raw_uri_pattern).content
buffer[:10]

In [ ]:
import io
import urllib
import zipfile

# retrieve list of URLs from the webservers
response = urllib.request.urlopen(raw_uri)
zippedData = response.read()

# myzipfile = zipfile.ZipFile(StringIO(zippedData))
myzipfile = zipfile.ZipFile(io.BytesIO(zippedData))

foofile = myzipfile.open('epicurve_symptom_date.csv')

In [ ]:
df = pandas.read_csv(foofile, parse_dates=['symptom date'])
df

In [ ]:
df = df[df.measure == 'state_total'][['symptom date', 'cases', 'deaths']].copy()
df.columns = ['Date', 'Cases', 'Deaths']
df = df.set_index('Date')
df